# **IF3170 Artificial Intelligence | Tugas Besar 2**

Group Number: xx

Group Members:
- Name (NIM)
- Name (NIM)
- ...

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier
from sklearn.base import BaseEstimator, TransformerMixin
from imblearn.under_sampling import RandomUnderSampler
from sklearn.preprocessing import QuantileTransformer
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline

## Import Dataset

In [2]:
# Import dataset from the training folder
df_additional = pd.read_csv('../dataset/train/additional_features_train.csv')
df_basic = pd.read_csv('../dataset/train/basic_features_train.csv')
df_content = pd.read_csv('../dataset/train/content_features_train.csv')
df_flow = pd.read_csv('../dataset/train/flow_features_train.csv')
df_labels = pd.read_csv('../dataset/train/labels_train.csv')
df_time = pd.read_csv('../dataset/train/time_features_train.csv')

In [3]:
# Join the datasets based on the 'id' attribute
df_merged = (
    df_time
    .merge(df_labels, on = "id", how = "left")
    .merge(df_flow, on = "id", how = "left")
    .merge(df_content, on = "id", how = "left")
    .merge(df_basic, on = "id", how = "left")
    .merge(df_additional, on = "id", how = "left")
)

df_merged.head()

,sjit,djit,sinpkt,dinpkt,tcprtt,synack,ackdat,id,attack_cat,label,...,ct_flw_http_mthd,is_ftp_login,ct_ftp_cmd,ct_srv_src,ct_srv_dst,ct_dst_ltm,ct_src_ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm
0,4449.110313,3234.831566,11.845558,6.261361,NaN,0.000444,0.000114,0,Normal,0,...,0.0,0.0,0.0,11.0,NaN,5.0,4.0,2.0,1.0,5.0
1,0.000000,0.000000,0.009000,0.000000,0.000000,0.000000,NaN,1,Generic,1,...,0.0,0.0,0.0,10.0,10.0,10.0,10.0,NaN,10.0,10.0
2,8561.040438,249.950547,165.386453,172.345750,0.158826,0.057902,0.100924,2,Exploits,1,...,0.0,0.0,0.0,4.0,4.0,2.0,2.0,1.0,1.0,4.0
3,4053.086020,2918.730804,8.669644,4.496707,0.000558,0.000448,NaN,3,Normal,0,...,0.0,0.0,0.0,9.0,9.0,3.0,2.0,2.0,1.0,6.0
4,0.000000,0.000000,0.008000,0.007000,0.000000,0.000000,0.000000,4,Normal,0,...,0.0,0.0,0.0,3.0,3.0,4.0,3.0,1.0,NaN,1.0


# Exploratory Data Analysis (Optional)

Exploratory Data Analysis (EDA) is a crucial step in the data analysis process that involves examining and visualizing data sets to uncover patterns, trends, anomalies, and insights. It is the first step before applying more advanced statistical and machine learning techniques. EDA helps you to gain a deep understanding of the data you are working with, allowing you to make informed decisions and formulate hypotheses for further analysis.

In [265]:
# Write your code here

# 1. Split Training Set and Validation Set

Splitting the training and validation set works as an early diagnostic towards the performance of the model we train. This is done before the preprocessing steps to **avoid data leakage inbetween the sets**. If you want to use k-fold cross-validation, split the data later and do the cleaning and preprocessing separately for each split.

Note: For training, you should use the data contained in the `train` folder given by the TA. The `test` data is only used for kaggle submission.

In [4]:
train_set, val_set = train_test_split(df_merged, test_size=0.2, random_state=42)

print(f"Training set size: {train_set.shape[0]}")
print(f"Validation set size: {val_set.shape[0]}")

Training set size: 140272
Validation set size: 35069


# 2. Data Cleaning and Preprocessing

This step is the first thing to be done once a Data Scientist have grasped a general knowledge of the data. Raw data is **seldom ready for training**, therefore steps need to be taken to clean and format the data for the Machine Learning model to interpret.

By performing data cleaning and preprocessing, you ensure that your dataset is ready for model training, leading to more accurate and reliable machine learning results. These steps are essential for transforming raw data into a format that machine learning algorithms can effectively learn from and make predictions.

We will give some common methods for you to try, but you only have to **at least implement one method for each process**. For each step that you will do, **please explain the reason why did you do that process. Write it in a markdown cell under the code cell you wrote.**

## A. Data Cleaning

**Data cleaning** is the crucial first step in preparing your dataset for machine learning. Raw data collected from various sources is often messy and may contain errors, missing values, and inconsistencies. Data cleaning involves the following steps:

1. **Handling Missing Data:** Identify and address missing values in the dataset. This can include imputing missing values, removing rows or columns with excessive missing data, or using more advanced techniques like interpolation.

2. **Dealing with Outliers:** Identify and handle outliers, which are data points significantly different from the rest of the dataset. Outliers can be removed or transformed to improve model performance.

3. **Data Validation:** Check for data integrity and consistency. Ensure that data types are correct, categorical variables have consistent labels, and numerical values fall within expected ranges.

4. **Removing Duplicates:** Identify and remove duplicate rows, as they can skew the model's training process and evaluation metrics.

5. **Feature Engineering**: Create new features or modify existing ones to extract relevant information. This step can involve scaling, normalizing, or encoding features for better model interpretability.

### I. Handling Missing Data

Missing data can adversely affect the performance and accuracy of machine learning models. There are several strategies to handle missing data in machine learning:

1. **Data Imputation:**

    a. **Mean, Median, or Mode Imputation:** For numerical features, you can replace missing values with the mean, median, or mode of the non-missing values in the same feature. This method is simple and often effective when data is missing at random.

    b. **Constant Value Imputation:** You can replace missing values with a predefined constant value (e.g., 0) if it makes sense for your dataset and problem.

    c. **Imputation Using Predictive Models:** More advanced techniques involve using predictive models to estimate missing values. For example, you can train a regression model to predict missing numerical values or a classification model to predict missing categorical values.

2. **Deletion of Missing Data:**

    a. **Listwise Deletion:** In cases where the amount of missing data is relatively small, you can simply remove rows with missing values from your dataset. However, this approach can lead to a loss of valuable information.

    b. **Column (Feature) Deletion:** If a feature has a large number of missing values and is not critical for your analysis, you can consider removing that feature altogether.

3. **Domain-Specific Strategies:**

    a. **Domain Knowledge:** In some cases, domain knowledge can guide the imputation process. For example, if you know that missing values are related to a specific condition, you can impute them accordingly.

4. **Imputation Libraries:**

    a. **Scikit-Learn:** Scikit-Learn provides a `SimpleImputer` class that can handle basic imputation strategies like mean, median, and mode imputation.

    b. **Fancyimpute:** Fancyimpute is a Python library that offers more advanced imputation techniques, including matrix factorization, k-nearest neighbors, and deep learning-based methods.

The choice of imputation method should be guided by the nature of your data, the amount of missing data, the problem you are trying to solve, and the assumptions you are willing to make.

To check if we could use the methods in the Deletion of Missing Data strategy, we first need to find how many of the data entries has missing values:

In [7]:
# Check the number of data entries with missing values
missing_data = train_set[train_set.isnull().any(axis=1)]
print(f"Number of entries in training set: {train_set.shape[0]}")
print(f"Number of entries in training set that has missing values: {missing_data.shape[0]}")

missing_data = val_set[val_set.isnull().any(axis=1)]
print(f"Number of entries in validation set: {val_set.shape[0]}")
print(f"Number of entries in validation set that has missing values: {val_set.shape[0]}")

Number of entries in training set: 140272
Number of entries in training set that has missing values: 123176
Number of entries in validation set: 35069
Number of entries in validation set that has missing values: 35069


As seen above, the number of entries that has missing values largely represents the original training dataset (that is, 87,8% of the training dataset has missing values), so we definitely cannot use the strategy Deletion of Missing Data with the method Listwise Deletion. Next, to check if we can use the method Column (Feature) Deletion, we check if there are some features that particularly have very many null values compared to the other features:

In [10]:
missing_values_count = train_set.isnull().sum()
print("Count for missing values in each feature in training set:")
print(missing_values_count)

missing_values_count = val_set.isnull().sum()
print("\nCount for missing values in each feature in validation set:")
print(missing_values_count)

Count for missing values in each feature in training set:
sjit                 7015
djit                 7109
sinpkt               6953
dinpkt               6999
tcprtt               7093
synack               6943
ackdat               6886
id                      0
attack_cat              0
label                   0
proto                7033
swin                 6971
dwin                 6999
stcpb                6941
dtcpb                7072
smean                7008
dmean                7078
trans_depth          7001
response_body_len    6999
state                7034
dur                  6957
sbytes               6836
dbytes               7069
sttl                 7022
dttl                 6937
sloss                7004
dloss                7209
service              7038
sload                7006
dload                7096
spkts                6959
dpkts                6918
is_sm_ips_ports      7028
ct_state_ttl         7000
ct_flw_http_mthd     6946
is_ftp_login         6910
ct_ftp

As seen above, the number of entries that has missing values is distributed pretty much evenly across the features except for the ID and target features, so we definitely cannot use the method Column (Feature) Deletion either.

Our option left is to do imputation on the training dataset. 

In [11]:
def impute(df: pd.DataFrame) -> pd.DataFrame:
    categorical_features = ['proto', 'state', 'service', 'is_sm_ips_ports', 'is_ftp_login', 'label', 'id']  # id is only used for join later
    target_feature = 'attack_cat'

    categorical_set = df[categorical_features]
    categorical_features.remove('id')
    numerical_set = df.drop(categorical_features, axis=1)
    numerical_set = numerical_set.drop(target_feature, axis=1)
    target_set = df[[target_feature, 'id']] # id is used for join later

    imputer_mode = SimpleImputer(strategy='most_frequent')
    imputer_median = SimpleImputer(strategy='median')

    categorical_set = pd.DataFrame(
        imputer_mode.fit_transform(categorical_set),
        columns=categorical_set.columns,
        index=categorical_set.index
    )

    numerical_set = pd.DataFrame(
        imputer_median.fit_transform(numerical_set),
        columns=numerical_set.columns,
        index=numerical_set.index
    )

    result = categorical_set.merge(numerical_set, on='id').merge(target_set, on='id')
    return result

In [12]:
train_set = impute(train_set)
val_set = impute(val_set)

missing_data_train = train_set[train_set.isnull().any(axis=1)]
missing_data_val = val_set[val_set.isnull().any(axis=1)]

print(f"Number of data entries in training set with missing values: {missing_data_train.shape[0]}")
print(f"Number of data entries in validation set with missing values: {missing_data_val.shape[0]}")

Number of data entries in training set with missing values: 0
Number of data entries in validation set with missing values: 0


### II. Dealing with Outliers

Outliers are data points that significantly differ from the majority of the data. They can be unusually high or low values that do not fit the pattern of the rest of the dataset. Outliers can significantly impact model performance, so it is important to handle them properly.

Some methods to handle outliers:
1. **Imputation**: Replace with mean, median, or a boundary value.
2. **Clipping**: Cap values to upper and lower limits.
3. **Transformation**: Use log, square root, or power transformations to reduce their influence.
4. **Model-Based**: Use algorithms robust to outliers (e.g., tree-based models, Huber regression).

In [14]:
def select_outliers(df: pd.DataFrame) -> pd.DataFrame:
    outliers_mask = pd.DataFrame(False, index=df.index, columns=df.columns)

    for column in df.select_dtypes(include=['number']).columns:
        Q1 = df[column].quantile(0.25)
        Q3 = df[column].quantile(0.75)
        IQR = Q3 - Q1

        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR

        outliers_mask[column] = (df[column] < lower_bound) | (df[column] > upper_bound)

    outlier_rows = outliers_mask.any(axis=1)
    return outlier_rows

In [ ]:
outliers_train = select_outliers(train_set)
outliers_val = select_outliers(val_set)

print(f"Number of outliers in the training set: {outliers_train.sum()}")
print(f"Number of outliers in the validation set: {outliers_val.sum()}")

Number of outliers in the training set: 115228
Number of outliers in the validation set: 28855
Number of data entries in training set with missing values: 0
Number of data entries in validation set with missing values: 0


In [17]:
def replace_outliers_with_median(df):
    for column in df.select_dtypes(include=['number']).columns:
        Q1 = df[column].quantile(0.25)
        Q3 = df[column].quantile(0.75)
        IQR = Q3 - Q1

        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR

        median = df[column].median()
        df[column] = df[column].apply(
            lambda x: median if x < lower_bound or x > upper_bound else x
        )
    return df

In [ ]:
train_set = replace_outliers_with_median(train_set)
val_set = replace_outliers_with_median(val_set)

outliers_train = select_outliers(train_set)
outliers_val = select_outliers(val_set)

print(f"Number of outliers in the training set: {outliers_train.sum()}")
print(f"Number of outliers in the validation set: {outliers_val.sum()}")

Number of outliers in the training set: 115677
Number of outliers in the validation set: 28851
Number of data entries in training set with missing values: 0
Number of data entries in validation set with missing values: 0


### III. Remove Duplicates
Handling duplicate values is crucial because they can compromise data integrity, leading to inaccurate analysis and insights. Duplicate entries can bias machine learning models, causing overfitting and reducing their ability to generalize to new data. They also inflate the dataset size unnecessarily, increasing computational costs and processing times. Additionally, duplicates can distort statistical measures and lead to inconsistencies, ultimately affecting the reliability of data-driven decisions and reporting. Ensuring data quality by removing duplicates is essential for accurate, efficient, and consistent analysis.

In [19]:
duplicates_count_train = train_set.duplicated(subset=[col for col in train_set.columns if col != 'id']).sum()
duplicates_count_val = val_set.duplicated(subset=[col for col in train_set.columns if col != 'id']).sum()

print(f"Number of duplicates in training dataset: {duplicates_count_train}")
print(f"Number of duplicates in validation dataset: {duplicates_count_val}")

Number of duplicates in training dataset: 28681
Number of duplicates in validation dataset: 5144


In [ ]:
train_set = train_set.drop_duplicates(subset=[col for col in train_set.columns if col != 'id'], keep='first')
val_set = val_set.drop_duplicates(subset=[col for col in train_set.columns if col != 'id'], keep='first')

duplicates_count_train = train_set.duplicated(subset=[col for col in train_set.columns if col != 'id']).sum()
duplicates_count_val = val_set.duplicated(subset=[col for col in train_set.columns if col != 'id']).sum()

print(f"Number of duplicates in training dataset: {duplicates_count_train}")
print(f"Number of duplicates in validation dataset: {duplicates_count_val}")

Number of duplicates in training dataset: 0
Number of duplicates in validation dataset: 0
Number of data entries in training set with missing values: 0
Number of data entries in validation set with missing values: 0


### IV. Feature Engineering

**Feature engineering** involves creating new features (input variables) or transforming existing ones to improve the performance of machine learning models. Feature engineering aims to enhance the model's ability to learn patterns and make accurate predictions from the data. It's often said that "good features make good models."

1. **Feature Selection:** Feature engineering can involve selecting the most relevant and informative features from the dataset. Removing irrelevant or redundant features not only simplifies the model but also reduces the risk of overfitting.

2. **Creating New Features:** Sometimes, the existing features may not capture the underlying patterns effectively. In such cases, engineers create new features that provide additional information. For example:
   
   - **Polynomial Features:** Engineers may create new features by taking the square, cube, or other higher-order terms of existing numerical features. This can help capture nonlinear relationships.
   
   - **Interaction Features:** Interaction features are created by combining two or more existing features. For example, if you have features "length" and "width," you can create an "area" feature by multiplying them.

3. **Binning or Discretization:** Continuous numerical features can be divided into bins or categories. For instance, age values can be grouped into bins like "child," "adult," and "senior."

4. **Domain-Specific Feature Engineering:** Depending on the domain and problem, engineers may create domain-specific features. For example, in fraud detection, features related to transaction history and user behavior may be engineered to identify anomalies.

Feature engineering is both a creative and iterative process. It requires a deep understanding of the data, domain knowledge, and experimentation to determine which features will enhance the model's predictive power.

Note: Our group decided to do feature selection **after one-hot encoding**.

One of the most common method for feature engineering is feature selection, which is selecting only the most impactful features from the entire dataset (that is, features that has a huge impact on improving the model's performance).

However, to do feature selection we usually need to use a model for deciding which features to keep, and these models cannot receive non numerical attributes. Therefore, we decided to push feature selection further down until one-hot encoding is done to the dataset so that we can use the models for feature selection. For now, we will only drop the 'id' attribute which we can be sure it will not help our model to predict the target class.

In [277]:
train_set = train_set.drop('id', axis=1)
val_set = val_set.drop('id', axis=1)

## B. Data Preprocessing

**Data preprocessing** is a broader step that encompasses both data cleaning and additional transformations to make the data suitable for machine learning algorithms. Its primary goals are:

1. **Feature Scaling:** Ensure that numerical features have similar scales. Common techniques include Min-Max scaling (scaling to a specific range) or standardization (mean-centered, unit variance).

2. **Encoding Categorical Variables:** Machine learning models typically work with numerical data, so categorical variables need to be encoded. This can be done using one-hot encoding, label encoding, or more advanced methods like target encoding.

3. **Handling Imbalanced Classes:** If dealing with imbalanced classes in a binary classification task, apply techniques such as oversampling, undersampling, or using different evaluation metrics to address class imbalance.

4. **Dimensionality Reduction:** Reduce the number of features using techniques like Principal Component Analysis (PCA) or feature selection to simplify the model and potentially improve its performance.

5. **Normalization:** Normalize data to achieve a standard distribution. This is particularly important for algorithms that assume normally distributed data.

### Notes on Preprocessing processes

It is advised to create functions or classes that have the same/similar type of inputs and outputs, so you can add, remove, or swap the order of the processes easily. You can implement the functions or classes by yourself

or

use `sklearn` library. To create a new preprocessing component in `sklearn`, implement a corresponding class that includes:
1. Inheritance to `BaseEstimator` and `TransformerMixin`
2. The method `fit`
3. The method `transform`

### I. Feature Scaling

**Feature scaling** is a preprocessing technique used in machine learning to standardize the range of independent variables or features of data. The primary goal of feature scaling is to ensure that all features contribute equally to the training process and that machine learning algorithms can work effectively with the data.

Here are the main reasons why feature scaling is important:

1. **Algorithm Sensitivity:** Many machine learning algorithms are sensitive to the scale of input features. If the scales of features are significantly different, some algorithms may perform poorly or take much longer to converge.

2. **Distance-Based Algorithms:** Algorithms that rely on distances or similarities between data points, such as k-nearest neighbors (KNN) and support vector machines (SVM), can be influenced by feature scales. Features with larger scales may dominate the distance calculations.

3. **Regularization:** Regularization techniques, like L1 (Lasso) and L2 (Ridge) regularization, add penalty terms based on feature coefficients. Scaling ensures that all features are treated equally in the regularization process.

Common methods for feature scaling include:

1. **Min-Max Scaling (Normalization):** This method scales features to a specific range, typically [0, 1]. It's done using the following formula:

   $$X' = \frac{X - X_{min}}{X_{max} - X_{min}}$$

   - Here, $X$ is the original feature value, $X_{min}$ is the minimum value of the feature, and $X_{max}$ is the maximum value of the feature.  
<br />
<br />
2. **Standardization (Z-score Scaling):** This method scales features to have a mean (average) of 0 and a standard deviation of 1. It's done using the following formula:

   $$X' = \frac{X - \mu}{\sigma}$$

   - $X$ is the original feature value, $\mu$ is the mean of the feature, and $\sigma$ is the standard deviation of the feature.  
<br />
<br />
3. **Robust Scaling:** Robust scaling is a method that scales features to the interquartile range (IQR) and is less affected by outliers. It's calculated as:

   $$X' = \frac{X - Q1}{Q3 - Q1}$$

   - $X$ is the original feature value, $Q1$ is the first quartile (25th percentile), and $Q3$ is the third quartile (75th percentile) of the feature.  
<br />
<br />
4. **Log Transformation:** In cases where data is highly skewed or has a heavy-tailed distribution, taking the logarithm of the feature values can help stabilize the variance and improve scaling.

The choice of scaling method depends on the characteristics of your data and the requirements of your machine learning algorithm. **Min-max scaling and standardization are the most commonly used techniques and work well for many datasets.**

Scaling should be applied separately to each training and test set to prevent data leakage from the test set into the training set. Additionally, **some algorithms may not require feature scaling, particularly tree-based models.**

In [ ]:
class MinMaxScaler(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.min_values = None
        self.max_values = None

    def fit(self, X, y=None):
        X = pd.DataFrame(X)
        numeric_columns = X.select_dtypes(include=['number'])
        self.numeric_features = numeric_columns.columns[numeric_columns.nunique() > 2]
        self.min_values = X[self.numeric_features].min()
        self.max_values = X[self.numeric_features].max()
        
        return self

    def transform(self, X):
        X = pd.DataFrame(X).copy()
        X[self.numeric_features] = (
            (X[self.numeric_features] - self.min_values) / 
            (self.max_values - self.min_values)
        )
        return X

Number of data entries in training set with missing values: 0
Number of data entries in validation set with missing values: 0


### II. Feature Encoding

**Feature encoding**, also known as **categorical encoding**, is the process of converting categorical data (non-numeric data) into a numerical format so that it can be used as input for machine learning algorithms. Most machine learning models require numerical data for training and prediction, so feature encoding is a critical step in data preprocessing.

Categorical data can take various forms, including:

1. **Nominal Data:** Categories with no intrinsic order, like colors or country names.  

2. **Ordinal Data:** Categories with a meaningful order but not necessarily equidistant, like education levels (e.g., "high school," "bachelor's," "master's").

There are several common methods for encoding categorical data:

1. **Label Encoding:**

   - Label encoding assigns a unique integer to each category in a feature.
   - It's suitable for ordinal data where there's a clear order among categories.
   - For example, if you have an "education" feature with values "high school," "bachelor's," and "master's," you can encode them as 0, 1, and 2, respectively.
<br />
<br />
2. **One-Hot Encoding:**

   - One-hot encoding creates a binary (0 or 1) column for each category in a nominal feature.
   - It's suitable for nominal data where there's no inherent order among categories.
   - Each category becomes a new feature, and the presence (1) or absence (0) of a category is indicated for each row.
<br />
<br />
3. **Target Encoding (Mean Encoding):**

   - Target encoding replaces each category with the mean of the target variable for that category.
   - It's often used for classification problems.

In [22]:
class OneHotEncoder(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.non_numeric_columns = {}

    def fit(self, X, y=None):
        df = pd.DataFrame(X)
        df = df.apply(pd.to_numeric, errors='ignore')
        non_numeric_columns = df.select_dtypes(exclude='number')
        self.non_numeric_columns = non_numeric_columns.columns
        
        return self

    def transform(self, X):
        df = pd.DataFrame(X)
        df_encoded = pd.get_dummies(df, columns=self.non_numeric_columns)

        return df_encoded

### Feature Selection

In [ ]:
class FeatureSelector(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.selector = SelectFromModel(RandomForestClassifier(n_estimators=100))

    def fit(self, X, y=None):
        self.y = y
        return self

    def transform(self, X):
        X_selected_array = self.selector.fit_transform(X, self.y)
        X_selected = pd.DataFrame(X_selected_array, columns=X.columns[self.selector.get_support()])

        return X_selected

### III. Handling Imbalanced Dataset

**Handling imbalanced datasets** is important because imbalanced data can lead to several issues that negatively impact the performance and reliability of machine learning models. Here are some key reasons:

1. **Biased Model Performance**:

 - Models trained on imbalanced data tend to be biased towards the majority class, leading to poor performance on the minority class. This can result in misleading accuracy metrics.

2. **Misleading Accuracy**:

 - High overall accuracy can be misleading in imbalanced datasets. For example, if 95% of the data belongs to one class, a model that always predicts the majority class will have 95% accuracy but will fail to identify the minority class.

3. **Poor Generalization**:

 - Models trained on imbalanced data may not generalize well to new, unseen data, especially if the minority class is underrepresented.


Some methods to handle imbalanced datasets:
1. **Resampling Methods**:

 - Oversampling: Increase the number of instances in the minority class by duplicating or generating synthetic samples (e.g., SMOTE).
 - Undersampling: Reduce the number of instances in the majority class to balance the dataset.

2. **Evaluation Metrics**:

 - Use appropriate evaluation metrics such as precision, recall, F1-score, ROC-AUC, and confusion matrix instead of accuracy to better assess model performance on imbalanced data.

3. **Algorithmic Approaches**:

 - Use algorithms that are designed to handle imbalanced data, such as decision trees, random forests, or ensemble methods.
 - Adjust class weights in algorithms to give more importance to the minority class.

In [41]:
class Undersampler(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.undersampler = RandomUnderSampler(random_state=42)
        self.y_resampled = None

    def fit(self, X, y):
        _, self.y_resampled = self.undersampler.fit_resample(X, y)
        return self

    def transform(self, X):
        X_resampled, y_resampled = self.undersampler.fit_resample(X, self.y_resampled)
        return X_resampled, y_resampled

### IV. Data Normalization

Data normalization is used to achieve a standard distribution. Without normalization, models or processes that rely on the assumption of normality may not work correctly. Normalization helps reduce the magnitude effect and ensures numerical stability during optimization.

In [ ]:
class DataNormalizer(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.transformer = QuantileTransformer(output_distribution='normal')
        self.selected_columns = None

    def fit(self, X, y):
        numeric_columns = X.select_dtypes(include='number')
        binary_columns = numeric_columns.columns[(numeric_columns.nunique() == 2)]
        self.selected_columns = numeric_columns.drop(columns=binary_columns).columns
        X[self.selected_columns] = X[self.selected_columns] + 1e-6
        self.transformer.fit(X[self.selected_columns])
        return self

    def transform(self, X):
        X = pd.DataFrame(X).copy()
        X[self.selected_columns] = self.transformer.transform(X[self.selected_columns])
        return X

### V. Dimensionality Reduction

Dimensionality reduction is a technique used in data preprocessing to reduce the number of input features (dimensions) in a dataset while retaining as much important information as possible. It is essential when dealing with high-dimensional data, where too many features can cause problems like increased computational costs, overfitting, and difficulty in visualization. Reducing dimensions simplifies the data, making it easier to analyze and improving the performance of machine learning models.

One of the main approaches to dimensionality reduction is feature extraction. Feature extraction creates new, smaller sets of features that capture the essence of the original data. Common techniques include:

1. **Principal Component Analysis (PCA)**: Converts correlated features into a smaller number of uncorrelated "principal components."
2. **t-SNE (t-Distributed Stochastic Neighbor Embedding)**: A visualization-focused method to project high-dimensional data into 2D or 3D spaces.
3. **Autoencoders**: Neural networks that learn compressed representations of the data.

In [ ]:
class DimensionalityReducer(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.pca = PCA(n_components=2)

    def fit(self, X, y=None):
        numeric_columns = X.select_dtypes(include='number')
        binary_columns = numeric_columns.columns[(numeric_columns.nunique() == 2)]
        self.selected_columns = numeric_columns.drop(columns=binary_columns)
        
        return self

    def transform(self, X):
        df = pd.DataFrame(X)
        df = df.dropna()
        result = self.pca.fit_transform(df[self.selected_columns.columns])
        result_df = pd.DataFrame(result, columns=[f'PC{i+1}' for i in range(result.shape[1])], index=df.index)
        non_numeric_columns = df.drop(columns=self.selected_columns)
        result_df = pd.concat([non_numeric_columns, result_df], axis=1)

        return result_df

# 3. Compile Preprocessing Pipeline

All of the preprocessing classes or functions defined earlier will be compiled in this step.

If you use sklearn to create preprocessing classes, you can list your preprocessing classes in the Pipeline object sequentially, and then fit and transform your data.

In [43]:
pipe = Pipeline([("scaler", MinMaxScaler()),
                 ("encoder", OneHotEncoder()),
                 ("normalizer", DataNormalizer()),
                 ("reducer", DimensionalityReducer()),
                 ("sampler", Undersampler())])

In [ ]:
x_train = train_set.drop('attack_cat', axis=1)
y_train = train_set['attack_cat']

x_val = val_set.drop('attack_cat', axis=1)
y_val = val_set['attack_cat']

x_train_processed, y_train_processed = pipe.fit(x_train, y_train).transform(x_train)
x_val_processed, y_val_processed = pipe.fit(x_val, y_val).transform(x_val)

# Hasil akhir preprocessing
train_set = pd.concat([x_train_processed, y_train_processed], axis=1)
val_set = pd.concat([x_val_processed, y_val_processed], axis=1)

missing_data_train = train_set[train_set.isnull().any(axis=1)]
missing_data_val = val_set[val_set.isnull().any(axis=1)]

print(f"Number of data entries in training set with missing values: {missing_data_train.shape[0]}")
print(f"Number of data entries in validation set with missing values: {missing_data_val.shape[0]}")

C:\Users\irvan\AppData\Local\Temp\ipykernel_3168\2366003513.py:7: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df = df.apply(pd.to_numeric, errors='ignore')
C:\Users\irvan\AppData\Local\Temp\ipykernel_3168\2366003513.py:7: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df = df.apply(pd.to_numeric, errors='ignore')


Number of data entries in training set with missing values: 0
Number of data entries in validation set with missing values: 0


c:\Users\irvan\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\preprocessing\_data.py:2785: UserWarning: n_quantiles (1000) is greater than the total number of samples (280). n_quantiles is set to n_samples.
  warnings.warn(


# 4. Modeling and Validation

Modelling is the process of building your own machine learning models to solve specific problems, or in this assignment context, predicting the target feature `attack_cat`. Validation is the process of evaluating your trained model using the validation set or cross-validation method and providing some metrics that can help you decide what to do in the next iteration of development.

## A. KNN

In [286]:
# Type your code here

## B. Naive Bayes

In [287]:
# Type your code here

## C. ID3

In [288]:
# Type your code here

## D. Improvements (Optional)

- **Visualize the model evaluation result**

This will help you to understand the details more clearly about your model's performance. From the visualization, you can see clearly if your model is leaning towards a class than the others. (Hint: confusion matrix, ROC-AUC curve, etc.)

- **Explore the hyperparameters of your models**

Each models have their own hyperparameters. And each of the hyperparameter have different effects on the model behaviour. You can optimize the model performance by finding the good set of hyperparameters through a process called **hyperparameter tuning**. (Hint: Grid search, random search, bayesian optimization)

- **Cross-validation**

Cross-validation is a critical technique in machine learning and data science for evaluating and validating the performance of predictive models. It provides a more **robust** and **reliable** evaluation method compared to a hold-out (single train-test set) validation. Though, it requires more time and computing power because of how cross-validation works. (Hint: k-fold cross-validation, stratified k-fold cross-validation, etc.)

In [289]:
# Type your code here

## E. Submission
To predict the test set target feature and submit the results to the kaggle competition platform, do the following:
1. Create a new pipeline instance identical to the first in Data Preprocessing
2. With the pipeline, apply `fit_transform` to the original training set before splitting, then only apply `transform` to the test set.
3. Retrain the model on the preprocessed training set
4. Predict the test set
5. Make sure the submission contains the `id` and `attack_cat` column.

In [290]:
# Type your code here

# 6. Error Analysis

Based on all the process you have done until the modeling and evaluation step, write an analysis to support each steps you have taken to solve this problem. Write the analysis using the markdown block. Some questions that may help you in writing the analysis:

- Does my model perform better in predicting one class than the other? If so, why is that?
- To each models I have tried, which performs the best and what could be the reason?
- Is it better for me to impute or drop the missing data? Why?
- Does feature scaling help improve my model performance?
- etc...

`Provide your analysis here`